<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/BelEvSer_NLP_HW_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=3c40460b615171527e1bbf540ea899bf1d1325ae17ee68a12677bb4fe00b71e6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
!pip install stop_words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import numpy as np


     

In [5]:
from google.colab import files
uploaded = files.upload()

Saving отзывы за лето.xls to отзывы за лето.xls


In [104]:
data =pd.read_excel('./отзывы за лето.xls')
data

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [105]:
max_words = 200
max_len = 200
num_classes = 1
# Training
epochs = 20
batch_size = 512
print_batch_n = 100
vec_len = 10

In [106]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

data['Content'] = data['Content'].apply(preprocess_text)


In [10]:
data.head()

,Rating,Content,Date
0,5,it just works,2017-08-14
1,4,целое удобноной приложениеиз минус хотеть боль...,2017-08-14
2,5,отлично,2017-08-14
3,5,зависать 1 работа антивирус ранее пользоваться...,2017-08-14
4,5,удобно работать быстро,2017-08-14


In [11]:
X_train, X_test, y_train, y_test = train_test_split(data.Content, data.Rating, test_size=0.33, random_state=42)


In [12]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [14]:
nltk.download("punkt")
tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]


In [17]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
     


In [20]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [21]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}


In [23]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [24]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)


In [25]:
x_test.shape


(6818, 200)

In [26]:
max_len

200

In [27]:
x_train[3]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 60], dtype=int32)

In [233]:
# Keras model

In [120]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, AveragePooling1D, Flatten
from keras.layers import  MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [29]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [31]:
import tensorflow as tf

In [32]:
num_classes = 6
y_train = tf.keras.utils.to_categorical(y_train['Rating'], num_classes)
y_test = tf.keras.utils.to_categorical(y_test['Rating'], num_classes)

In [35]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
     

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 23s 851ms/step - loss: 1.2815 - accuracy: 0.6956 - val_loss: 1.0709 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 23s 925ms/step - loss: 0.9448 - accuracy: 0.7039 - val_loss: 0.8856 - val_accuracy: 0.7040
Epoch 3/20
25/25 [==============================] - 23s 934ms/step - loss: 0.8134 - accuracy: 0.7082 - val_loss: 0.7956 - val_accuracy: 0.7047
Epoch 4/20
25/25 [==============================] - 23s 912ms/step - loss: 0.7406 - accuracy: 0.7428 - val_loss: 0.7440 - val_accuracy: 0.7466
Epoch 5/20
25/25 [==============================] - 21s 834ms/step - loss: 0.6952 - accuracy: 0.7712 - val_loss: 0.7264 - val_accuracy: 0.7509
Epoch 6/20
25/25 [==============================] - 22s 885ms/step - loss: 0.6726 - accuracy: 0.7758 - val_loss: 0.7218 - val_accuracy: 0.7509
Epoch 7/20
25/25 [==============================] - 22s 884ms/step - loss: 0.6579 - accuracy: 0.7791 - val_loss: 0.7131 - val_accuracy: 0.7531

In [38]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


14/14 [==============================] - 4s 297ms/step - loss: 0.6650 - accuracy: 0.7693


Test score: 0.6650132536888123
Test accuracy: 0.7692871689796448


In [39]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 3s 197ms/step


In [40]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
    

In [41]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
tensorboard1=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping1=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard1, early_stopping1])

Epoch 1/20
25/25 [==============================] - 23s 893ms/step - loss: 1.2383 - accuracy: 0.7021 - val_loss: 0.9638 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 22s 881ms/step - loss: 0.8545 - accuracy: 0.7201 - val_loss: 0.7529 - val_accuracy: 0.7379
Epoch 3/20
25/25 [==============================] - 21s 830ms/step - loss: 0.7126 - accuracy: 0.7600 - val_loss: 0.7172 - val_accuracy: 0.7451
Epoch 4/20
25/25 [==============================] - 22s 886ms/step - loss: 0.6856 - accuracy: 0.7665 - val_loss: 0.7085 - val_accuracy: 0.7495
Epoch 5/20
25/25 [==============================] - 22s 888ms/step - loss: 0.6723 - accuracy: 0.7701 - val_loss: 0.7078 - val_accuracy: 0.7466
Epoch 6/20
25/25 [==============================] - 21s 831ms/step - loss: 0.6620 - accuracy: 0.7760 - val_loss: 0.7056 - val_accuracy: 0.7444
Epoch 7/20
25/25 [==============================] - 22s 888ms/step - loss: 0.6515 - accuracy: 0.7783 - val_loss: 0.7037 - val_accuracy: 0.7495

In [43]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 3s 190ms/step - loss: 0.6615 - accuracy: 0.7705


Test score: 0.6614837050437927
Test accuracy: 0.7704605460166931


In [44]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 3s 195ms/step


In [234]:
#Word2Vec

In [45]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score
import multiprocessing


In [46]:
sentences_list_of_lists = [[sentence] for sentence in data['Content'].tolist()]
sentences_list_of_lists[:10]

[['it just works'],
 ['целое удобноной приложениеиз минус хотеть большой доступ персональный данные телефонеприходиться пользоваться ограниченный режим'],
 ['отлично'],
 ['зависать 1 работа антивирус ранее пользоваться нормально'],
 ['удобно работать быстро'],
 ['удобно норма 👍👍👍'],
 ['удобный приложение'],
 ['устраивать'],
 ['работать чётко отличие банкомат вечно зависать тупить'],
 ['хорошо👍']]

In [47]:
modelW2V = Word2Vec(sentences= X_train.apply(str.split), vector_size =100, window=5, min_count=5, workers=8)

In [48]:
modelW2V = Word2Vec(sentences=sentences_list_of_lists,
                    vector_size=vec_len,
                    window=10,
                    min_count=2,
                    workers=multiprocessing.cpu_count())

In [49]:
w = np.random.random((max_words, vec_len))
w

array([[0.39141157, 0.19974199, 0.23761134, ..., 0.53971908, 0.44802227,
        0.63835946],
       [0.42992454, 0.07458237, 0.55249839, ..., 0.30312624, 0.39101863,
        0.05168607],
       [0.93065366, 0.84089718, 0.69675814, ..., 0.38659648, 0.35505362,
        0.33761945],
       ...,
       [0.87800617, 0.04766849, 0.44011625, ..., 0.55950368, 0.39106839,
        0.16750504],
       [0.42380366, 0.59682715, 0.89670274, ..., 0.22329589, 0.99994124,
        0.92951205],
       [0.8676014 , 0.9301788 , 0.88149589, ..., 0.94526969, 0.34630894,
        0.17076389]])

In [50]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=vec_len, weights=[w], input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [52]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs = 80,
                    verbose=0,
                    validation_split=0.2,
                    )

In [53]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 1s 96ms/step - loss: 0.7648 - accuracy: 0.7589


Test score: 0.7647739052772522
Test accuracy: 0.7588735818862915


In [61]:
!pip install gensim --upgrade




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
from gensim.models import KeyedVectors

In [91]:
import io
import gzip
import pathlib
import urllib.request

WORD2VEC_PATH = pathlib.Path('word2vec.bin')

if not WORD2VEC_PATH.exists():
    url = 'https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz'
    with urllib.request.urlopen(url) as connection:
      opener = urllib.request.build_opener()
      opener.addheaders = [('User-Agent', 'MyApp/1.0')]
      urllib.request.install_opener(opener)
      urllib.request.urlretrieve('https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz')
      compressed = connection.read()
            
    decompressed = gzip.GzipFile(fileobj=io.BytesIO(compressed), mode='rb').read()
    WORD2VEC_PATH.write_bytes(decompressed)

HTTPError: ignored

In [92]:
!pip install --upgrade simple_elmo
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
ru_w2v = KeyedVectors.load_word2vec_format('https://rusvectores.org/static/models/ruwikiruscorpora_upos_cbow_300_10_2021.bin.gz', binary=True)

HTTPError: ignored

In [ ]:
#ссылки на модели не работают 

In [101]:
import gensim
import gensim.downloader as download_api
russian_model = download_api.load('word2vec-ruscorpora-300')

[=================================================-] 100.0% 198.8/198.8MB downloaded


In [102]:
def code_w2v_txt(txt, max_len = 100):
    sent_w2v = []
    zero_point = np.zeros(300)
    txt = txt.split()
   
    for i in range(max_len):
        try:
            word =russian_model[txt[i]]
        except:
            word = zero_point
        sent_w2v.append(word)
    return np.array(sent_w2v)

max_len = 50
data['w2v'] = data['Content'].apply(code_w2v_txt, max_len = max_len)
del(russian_model)
     

In [103]:
X = list(data['w2v'].values)
X = np.array(X)
y = data['Rating'].values

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [108]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_enc_labels = le.fit_transform(y_train) 
test_enc_labels = le.transform(y_test)
le.classes_
     

array([1, 2, 3, 4, 5])

In [110]:
from keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate

In [121]:
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
l_cov1= Conv1D(128, 5, activation='relu')(inputs)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 =Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(2)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(64, activation='softmax')(l_dense)
preds = Dense(num_classes, activation='softmax')(l_dense)

model = Model(inputs=inputs, outputs=preds)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
     

In [126]:
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
l_cov1= Conv1D(128, 3, activation='relu')(inputs)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 =Conv1D(128, 3, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(2)(l_cov2)
l_cov3 = Conv1D(128, 3, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(3)(l_cov3)  
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(64, activation='softmax')(l_dense)
preds = Dense(num_classes, activation='softmax')(l_dense)

model = Model(inputs=inputs, outputs=preds)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [127]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, train_enc_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 12456 samples, validate on 1385 samples
Epoch 1/20
12456/12456 [==============================] - 11s 869us/sample - loss: 1.5977 - accuracy: 0.6787 - val_loss: 1.5846 - val_accuracy: 0.7011
Epoch 2/20
12456/12456 [==============================] - 10s 843us/sample - loss: 1.5731 - accuracy: 0.7030 - val_loss: 1.5604 - val_accuracy: 0.7011
Epoch 3/20
12456/12456 [==============================] - 11s 887us/sample - loss: 1.5491 - accuracy: 0.7030 - val_loss: 1.5368 - val_accuracy: 0.7011
Epoch 4/20
12456/12456 [==============================] - 11s 887us/sample - loss: 1.5257 - accuracy: 0.7030 - val_loss: 1.5138 - val_accuracy: 0.7011
Epoch 5/20
12456/12456 [==============================] - 12s 970us/sample - loss: 1.5029 - accuracy: 0.7030 - val_loss: 1.4915 - val_accuracy: 0.7011
Epoch 6/20
12456/12456 [==============================] - 11s 900us/sample - loss: 1.4808 - accuracy: 0.7030 - val_loss: 1.4699 - val_accuracy: 0.7011
Epoch 7/20
12456/12456 [=====================

In [128]:
score = model.evaluate(X_test, test_enc_labels)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])
     



Test score: 1.2222793217662073
Test accuracy: 0.71252567


In [ ]:
#предобученные модели менее точные